(tune-mnist-keras)=

# Using Keras & TensorFlow with Tune

```{image} /images/tf_keras_logo.jpeg
:align: center
:alt: Keras & TensorFlow Logo
:height: 120px
:target: https://keras.io
```

```{contents}
:backlinks: none
:local: true
```

## Example

In [1]:
import argparse
import os

from filelock import FileLock
from tensorflow.keras.datasets import mnist

import ray
from ray import air, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback


def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    import tensorflow as tf

    batch_size = 128
    num_classes = 10
    epochs = 12

    with FileLock(os.path.expanduser("~/.data.lock")):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(config["hidden"], activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"],
    )

    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({"mean_accuracy": "accuracy"})],
    )


def tune_mnist(num_training_iterations):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(train_mnist, resources={"cpu": 2, "gpu": 0}),
        tune_config=tune.TuneConfig(
            metric="mean_accuracy",
            mode="max",
            scheduler=sched,
            num_samples=10,
        ),
        run_config=air.RunConfig(
            name="exp",
            stop={"mean_accuracy": 0.99, "training_iteration": num_training_iterations},
        ),
        param_space={
            "threads": 2,
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
            "hidden": tune.randint(32, 512),
        },
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing"
    )
    args, _ = parser.parse_known_args()
    if args.smoke_test:
        ray.init(num_cpus=4)

    tune_mnist(num_training_iterations=5 if args.smoke_test else 300)


2022-07-22 16:16:58,114	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8269
2022-07-22 16:17:00,822	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_55a9b_00000,TERMINATED,127.0.0.1:51968,276,0.0406397,0.817788,0.98455,12,78.3252
train_mnist_55a9b_00001,TERMINATED,127.0.0.1:51977,380,0.0873557,0.524634,0.983717,12,74.9888
train_mnist_55a9b_00002,TERMINATED,127.0.0.1:51984,258,0.0951813,0.825499,0.990417,11,64.1272
train_mnist_55a9b_00003,TERMINATED,127.0.0.1:51991,255,0.0971683,0.23161,0.977633,12,60.8475
train_mnist_55a9b_00004,TERMINATED,127.0.0.1:52000,303,0.00440117,0.325439,0.90775,12,55.5722
train_mnist_55a9b_00005,TERMINATED,127.0.0.1:52007,92,0.0651919,0.710183,0.974867,12,44.8092
train_mnist_55a9b_00006,TERMINATED,127.0.0.1:52016,211,0.0731116,0.127751,0.97025,12,42.1217
train_mnist_55a9b_00007,TERMINATED,127.0.0.1:52021,181,0.0362389,0.790345,0.979967,12,41.7632
train_mnist_55a9b_00008,TERMINATED,127.0.0.1:52007,142,0.0323741,0.660418,0.969367,12,14.1527
train_mnist_55a9b_00009,TERMINATED,127.0.0.1:51984,97,0.0244971,0.175045,0.9407,12,12.6405


2022-07-22 16:17:01,834	INFO plugin_schema_manager.py:52 -- Loading the default runtime env schemas: ['/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].
(train_mnist pid=51968) 2022-07-22 16:17:08.627419: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(train_mnist pid=51968) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(train_mnist pid=51968) /Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
(train_mnist pid=51968)   "The `lr` argument is deprecated, use `learning_rate` instead.")
(train_mni

Result for train_mnist_55a9b_00000:
  date: 2022-07-22_16-17-10
  done: false
  experiment_id: 3659349c38c746cfb71b4db5eb9302a0
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.8903833627700806
  node_ip: 127.0.0.1
  pid: 51968
  time_since_restore: 2.439258098602295
  time_this_iter_s: 2.439258098602295
  time_total_s: 2.439258098602295
  timestamp: 1658503030
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 55a9b_00000
  warmup_time: 0.003445863723754883
  
Result for train_mnist_55a9b_00004:
  date: 2022-07-22_16-17-33
  done: false
  experiment_id: 6eb62b7cb38f442a867a9094f0664701
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.6376166939735413
  node_ip: 127.0.0.1
  pid: 52000
  time_since_restore: 2.4364511966705322
  time_this_iter_s: 2.4364511966705322
  time_total_s: 2.4364511966705322
  timestamp: 1658503053
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 55a9b_00004
  warm

(train_mnist pid=52021) 2022-07-22 16:17:51.567914: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(train_mnist pid=52021) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(train_mnist pid=52021) /Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
(train_mnist pid=52021)   "The `lr` argument is deprecated, use `learning_rate` instead.")
(train_mnist pid=52021) 2022-07-22 16:17:52.977183: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Result for train_mnist_55a9b_00005:
  date: 2022-07-22_16-17-54
  done: false
  experiment_id: 8dbd22e6caed4fe39351dffa3ef14eac
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 3
  mean_accuracy: 0.9490833282470703
  node_ip: 127.0.0.1
  pid: 52007
  time_since_restore: 17.22033405303955
  time_this_iter_s: 2.672102928161621
  time_total_s: 17.22033405303955
  timestamp: 1658503074
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: 55a9b_00005
  warmup_time: 0.005449056625366211
  
Result for train_mnist_55a9b_00006:
  date: 2022-07-22_16-17-54
  done: false
  experiment_id: 9594405e38084311a891b48addd13f75
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 3
  mean_accuracy: 0.9327999949455261
  node_ip: 127.0.0.1
  pid: 52016
  time_since_restore: 11.758372068405151
  time_this_iter_s: 3.0426323413848877
  time_total_s: 11.758372068405151
  timestamp: 1658503074
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: 55a9b_00006
  warm

2022-07-22 16:18:36,803	INFO tune.py:738 -- Total run time: 95.98 seconds (95.03 seconds for the tuning loop).


Best hyperparameters found were:  {'threads': 2, 'lr': 0.09518133271957563, 'momentum': 0.8254987643140009, 'hidden': 258}


## More Keras and TensorFlow Examples

- {doc}`/tune/examples/includes/pbt_memnn_example`: Example of training a Memory NN on bAbI with Keras using PBT.
- {doc}`/tune/examples/includes/tf_mnist_example`: Converts the Advanced TF2.0 MNIST example to use Tune
  with the Trainable. This uses `tf.function`.
  Original code from tensorflow: https://www.tensorflow.org/tutorials/quickstart/advanced
- {doc}`/tune/examples/includes/pbt_tune_cifar10_with_keras`:
  A contributed example of tuning a Keras model on CIFAR10 with the PopulationBasedTraining scheduler.
